In [1]:
import datetime

In [3]:
MODEL_NAME = "all_leagues"
MODEL_VERSION = 1.0
MODEL_TYPE = "fulltime_win_outcome"

In [4]:
# UPDATING THE GITHUB OF THE DATASETS
MODEL_TIME = datetime.datetime.now().strftime("%d-%M-%Y_%T")
!git add .
!git commit -m "updates: {MODEL_NAME}_{MODEL_TYPE}-v{MODEL_VERSION}: bot commit - {MODEL_TIME}"
!git push

[main f1fa5a1] updates: all_leagues-1.0-vfulltime_win_outcome: bot commit - 23-18-2024_07:18:27
 3 files changed, 1784 insertions(+), 165 deletions(-)
fatal: unable to access 'https://github.com/lebyanelm/neural-trained-models.git/': Could not resolve host: github.com
